In [16]:
%matplotlib inline
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
from matplotlib.colors import SymLogNorm
import scienceplots

import astropy.units as u
from astropy import constants
from astropy.table import Table
from astropy.coordinates import SkyCoord,FK5,ICRS
from astropy.io import fits,ascii
from astropy.wcs import WCS, utils as wcsutils
from astropy.stats import sigma_clipped_stats
import astropy.visualization as vis
from astropy.modeling import models, fitting
from astropy.nddata import StdDevUncertainty, Cutout2D
from astropy.convolution import Gaussian2DKernel,Gaussian1DKernel
from astropy.cosmology import Planck18
from scipy.signal import find_peaks
#from scipy.constants import c



from photutils.aperture import EllipticalAperture, SkyEllipticalAperture

import os, sys, glob, pdb, scipy

from importlib import reload
import pyregion as pyreg
from spectral_cube import SpectralCube
from reproject import reproject_interp
import regions
from regions.shapes.circle import CircleSkyRegion,CirclePixelRegion
from regions import CircleAnnulusSkyRegion, CircleAnnulusPixelRegion, RectangleSkyRegion
from regions import PixCoord
sys.path.append('/disk/bifrost/yuanze/KBSS/MUSEQSO/scripts')
sys.path.append('/disk/bifrost/yuanze/KBSS/CubEx_run/scripts')
import run_cubetools_v1 as ctools
import makeMask_MUSE as mmask
reload(ctools)
reload(mmask)
sys.path.append('/disk/bifrost/yuanze/software/GalfitS/src')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import images as IM
import sed_interp
reload(IM)

brightu = r"Brightness $\rm (10^{-4}~erg~s^{-1}~arcsec^{-2}~cm^{-2}~\AA^{-1})$"
def imshow_astro(img, wcsinfo = None, figsize = (10,10), colorbar =True,
               cblabel="", cbfrac = 0.035, norm = None,
               stretch = vis.LinearStretch(), cmap = "hot",
               vrange = (None, None)):
    
    _, med, std = sigma_clipped_stats(img.data)
    
    fig =  plt.figure(figsize = figsize)
    if wcsinfo:
        ax = plt.subplot(projection = wcsinfo)
    else:
        ax = plt.subplot()
    
    vmin, vmax = vrange
    
    if not vmin:
        vmin = med
    if not vmax:
        vmax = med + 10*std
    if not norm:
        norm = vis.ImageNormalize(vmin = vmin, vmax = vmax, stretch = stretch)
    im = ax.imshow(img, norm = norm, cmap = cmap)
    if colorbar:
        cb = plt.colorbar(im, label = cblabel, fraction =cbfrac)
    return fig, ax, im
KBSSpath="/disk/bifrost/yuanze/KBSS"
source_table = ascii.read(KBSSpath+"/KCWI/KBSS_faint_AGN.list",format="ipac")

#MUSEQSO= ascii.read(KBSSpath+"/KCWI/MUSEQSO_machine_readable_updated2.list",format="ipac")
#museqso_path = os.path.join(KBSSpath, "MUSEQSO")
#votable_file = os.path.join("fixed_file.xml")

In [33]:

reload(mmask)
from datetime import datetime# Making white-light cutout
root_directory = KBSSpath
filters = ["table['KCWI'] == 'yes'"]
dtype="KBSS"
#condition=[qsos_bright['Field']!="Q1623",qsos_bright['Field']!="Q0142"]
all_directories,qsotab,all_data_dir = ctools.find_directories_from_ascii(source_table, root_directory,filters=filters,KBSS=(dtype=="KBSS"))
#source_table = qsos_bright#[(qsos_bright['Field']!="Q1623") & (qsos_bright['Field']!="Q0142")]

#print("Number of directories found:",len(all_directories))
#gc.set_debug(gc.DEBUG_LEAK)
#radius=6
dovar=True
for ne,element in enumerate(qsotab):#source_table[qsos_bright['Field']=="Q1623"]:
    #print(element)
    field = element["Field"]
    objname=element["Name"]
    cname=element["CName"]
    subpath=all_directories[ne]
    sub_data_path=all_data_dir[ne]
    #CubRunpath=KBSSpath+"/CubEx_run/"+field
    fn=subpath+"/{}-{}_icubes.PSFSub.fits".format(field,objname)
    hdu0=fits.open(fn)
    hdr = hdu0[0].header
    wc = (np.arange(hdr['NAXIS3']) - hdr['CRPIX3'] + 1) * hdr['CD3_3'] + hdr['CRVAL3']
    #NSubfile = adp_prefix+".PSFSub.fits"
    #zmask = np.loadtxt(subdir+"/zmask.txt").astype(np.int64)
    zmask = mmask.determine_mask(wc,"continuum", element['zlya'])
    xpix = element["x"]
    ypix = element["y"]
    wimage = np.nansum(hdu0[0].data[zmask[0]:zmask[1],:],axis=0)
    wimage[wimage==0] = np.nan
    header = fits.Header()
    header['DATE'] = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    header['CREATOR'] = 'Yuanze Ding'
    header['COMMENT'] = 'White-light cutout from 3D data cube'
    header['YSOURCE'] = ypix
    header['XSOURCE'] = xpix
    header['OBJECT'] = objname
    # Extract WCS information from the original header
    wcs_keys = ['CTYPE1', 'CTYPE2', 'CRVAL1', 'CRVAL2', 'CRPIX1', 'CRPIX2', 'CDELT1', 'CDELT2', 'CUNIT1', 'CUNIT2', 'CD1_1', 'CD1_2', 'CD2_1', 'CD2_2']
    for key in wcs_keys:
        if key in hdr:
            header[key] = hdr[key]

    hdu = fits.PrimaryHDU(wimage,header=header)
    output_fn = subpath+"/{}-{}_icubes.PSFSub.white.fits".format(field,objname)
    hdulist = fits.HDUList([hdu])
    print("writing:",output_fn)
    hdulist.writeto(output_fn, overwrite=True)
    if dovar:
        fnvar=sub_data_path+"/{}-{}_vcubes.fits".format(field,cname)
        hdu0_var=fits.open(fnvar)
        wvar = np.nansum(hdu0_var[0].data[zmask[0]:zmask[1],:],axis=0)
        wvar[wvar==0] = np.nan
        output_fnvar = subpath+"/{}-{}_icubes.whitevar.fits".format(field,objname)
        hduvar = fits.PrimaryHDU(wvar,header=header)
        hdulist = fits.HDUList([hduvar])
        hdulist.writeto(output_fnvar, overwrite=True)
        print("writing:",output_fnvar)
            #hdu2=fits.open(KBSSpath+"/MUSEQSO/Q-0347-383/ADP.2016-06-02T13:21:57.103.fits")

writing: /disk/bifrost/yuanze/KBSS/Q0100/BX160/kcwib/Q0100-BX160_icubes.PSFSub.white.fits
writing: /disk/bifrost/yuanze/KBSS/Q0100/BX160/kcwib/Q0100-BX160_icubes.whitevar.fits
writing: /disk/bifrost/yuanze/KBSS/Q0100/BX164/kcwib/Q0100-BX164_icubes.PSFSub.white.fits
writing: /disk/bifrost/yuanze/KBSS/Q0100/BX164/kcwib/Q0100-BX164_icubes.whitevar.fits
writing: /disk/bifrost/yuanze/KBSS/Q0105/BX58/kcwib/Q0105-BX58_icubes.PSFSub.white.fits
writing: /disk/bifrost/yuanze/KBSS/Q0105/BX58/kcwib/Q0105-BX58_icubes.whitevar.fits
writing: /disk/bifrost/yuanze/KBSS/Q0207/D9/kcwib/Q0207-D9_icubes.PSFSub.white.fits
writing: /disk/bifrost/yuanze/KBSS/Q0207/D9/kcwib/Q0207-D9_icubes.whitevar.fits
writing: /disk/bifrost/yuanze/KBSS/Q0256/C31/kcwib/Q0256-C31_icubes.PSFSub.white.fits
writing: /disk/bifrost/yuanze/KBSS/Q0256/C31/kcwib/Q0256-C31_icubes.whitevar.fits
writing: /disk/bifrost/yuanze/KBSS/Q0821/RK5/kcwib/Q0821-RK5_icubes.PSFSub.white.fits
writing: /disk/bifrost/yuanze/KBSS/Q0821/RK5/kcwib/Q0821-R

In [27]:
wc

array([3773.5, 3774. , 3774.5, ..., 4868. , 4868.5, 4869. ])

In [36]:
import warnings
from regions import Regions
reload(ctools)
root_directory = KBSSpath
filters = ["table['KCWI'] == 'yes'"]
dtype="KBSS"
#condition=[qsos_bright['Field']!="Q1623",qsos_bright['Field']!="Q0142"]
all_directories,qsotab,all_data_dir = ctools.find_directories_from_ascii(source_table, root_directory,filters=filters,KBSS=(dtype=="KBSS"))
print("Number of directories found:",len(all_directories))
#gc.set_debug(gc.DEBUG_LEAK)
#radius=6
cutr=20
mark_region=False
for ne,element in enumerate(qsotab):
    #qname = element["Name"]
    field = element["Field"]
    #print(qname)
    
    objname=element["Name"]
    cname=element["CName"]
    subpath=all_directories[ne]
    CubRunpath=subpath
    ra = element['RA']
    dec = element['Decl']
    sc= SkyCoord(ra,dec,unit=(u.deg,u.deg))
   # qname="LBQS 2139-4434"
    fname=subpath+"/{}-{}_icubes.PSFSub.white.fits".format(field,objname)
    fnamevar=subpath+"/{}-{}_icubes.whitevar.fits".format(field,objname)
    hduvar=fits.open(fnamevar)
    var=hduvar[0].data
    #var[var==0]=np.nan
    mean_sky_var, _, _ = sigma_clipped_stats(var, sigma=2.0, maxiters=None)
    img = IM.image(fname,hdu=0,unit=u.Jy)
    xpix = element["x"]
    ypix = element["y"]
    wcs=WCS(img.data.header)
    
    artists=None
    metas=None
    if mark_region:
        artists=[]
        metas=[]
        if os.path.exists(KBSSpath+f"/AbsHosts_nLOS_reg/{field}_AbsHost-nLOS.reg"):
            reg= ctools.read_regions(KBSSpath+f"/AbsHosts_nLOS_reg/{field}_AbsHost-nLOS.reg",pix_scale=wcs.pixel_scale_matrix[1,1]*3600)
            for r in reg:
                artists.append(r.to_pixel(wcs).as_artist())
                meta=dict(r.meta)
                meta["fontsize"]=5
                metas.append(meta)
    center = [ypix,xpix]
    imcut, cp = img.img_cut(sc.ra.value,sc.dec.value,cutr)
    sky_mean, sky_median, sky_std = sigma_clipped_stats(imcut, sigma=3.0, maxiters=None)
    print("sky level from vcube:",np.sqrt(mean_sky_var),"; from sigma clipping:",sky_std,"; bkg level:",sky_mean)
    img.sources_skycord_cut = [xpix,ypix]
    #img.cut_mask_image = np.zeros_like(img.cut_sigma_image)
    sdmask = ctools.create_circular_mask(img.data.shape, center, 7)
    try:
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always")
            img.cut_mask_image = img.generate_cutmask(4,nsigma=2,sky_level=np.sqrt(mean_sky_var),sdmask=sdmask,data = img.data.data,deblend=False,addgrow=1,nlevels=32,source_dia=5,contrast=0.001)
    except:
        for warning in w:
            print(f"Warning: {warning.message}")
        continue
    ctools.maskplot(img.data.data,img.cut_mask_image,meta=metas,artist=artists,output=root_directory+"/diagnostic/"+field+"-"+objname+"_maskmap.pdf",center=center)
    outname = subpath+"/{}-{}_icubes.PSFSub.mask.fits".format(field,objname)
    ctools.write_fits_cube(img.cut_mask_image.astype(np.int32),img.data.header,outname)
    #NSubfile = adp_prefix+".PSFSub.fits"
    #zmask = np.loadtxt(subdir+"/zmask.txt",dtype=int)

Number of directories found: 16
15.78142 13.28404
60.80073972954136 40.31546017677452
sky level from vcube: 0.027342526398253582 ; from sigma clipping: 0.029123653 ; bkg level: -0.0011673006


15.78195 13.27525
71.8315234556299 34.32593842984912
sky level from vcube: 0.02552316218263412 ; from sigma clipping: 0.027587565 ; bkg level: -0.0018367036


17.00674 16.5668
48.52490529011965 49.28110528403071
sky level from vcube: 0.028803707480054213 ; from sigma clipping: 0.028375285 ; bkg level: 0.006012119


32.43984 -0.13071
47.7884641358883 51.8275733186613
sky level from vcube: 0.02922133262294619 ; from sigma clipping: 0.023713559 ; bkg level: 0.006170804


44.741536 0.212853
47.43647333998302 49.916906695467176
sky level from vcube: 0.029564812522407415 ; from sigma clipping: 0.022549078 ; bkg level: 0.006328516


...

356.60088 12.80236
38.58340649798217 43.33426366636124
sky level from vcube: 0.028758970712109793 ; from sigma clipping: 0.026838116 ; bkg level: 0.008091365


356.50886 12.80739
55.39470920103242 56.69424412078324
sky level from vcube: 0.03518469145224817 ; from sigma clipping: 0.02776543 ; bkg level: 0.009925803


339.875266 11.789784
47.71215509841256 50.088908059579616
sky level from vcube: 0.027985096839131862 ; from sigma clipping: 0.022089984 ; bkg level: 0.0076515395


334.290161 0.300204
47.93647998817048 46.7955733632926
sky level from vcube: 0.03189696284015394 ; from sigma clipping: 0.024298657 ; bkg level: 0.0075872038


334.22233091 0.3450005
50.39752185768673 46.02024000506505
sky level from vcube: 0.04343800433871178 ; from sigma clipping: 0.05688532 ; bkg level: 0.07170146


{'label': 'Q0100  2.721'}

In [81]:
artists[0].center

(50.04648997238098, 49.072736870607734)

In [67]:
reg[0].meta

{'label': 'Q0100  2.721'}

In [53]:
import re
from astropy.coordinates import SkyCoord
from regions import CircleSkyRegion, PixCoord
from astropy import units as u

# Read the region file
file_path = KBSSpath+f"/AbsHosts_nLOS_reg/{field}_AbsHost-nLOS.reg"
with open(file_path, 'r') as file:
    lines = file.readlines()

# Regex to extract information
pattern = re.compile(r'fk5;circle\((.*?),(.*?),(.*?)i\) #color=(.*?)  width=2 text={(.*?)}')

regions = []

for line in lines:
    match = pattern.match(line)
    if match:
        ra, dec, radius, color, text = match.groups()
        coord = SkyCoord(ra, dec, unit=(u.hourangle, u.deg), frame='fk5')
        radius = float(radius) * u.arcsec  # assuming the radius is in arcseconds
        
        # Create a CircleSkyRegion
        region = CircleSkyRegion(center=coord, radius=radius)
        regions.append(region)

# Now `regions` contains all the parsed CircleSkyRegions
# You can manipulate them or plot them using regions library

# Example: Printing region details
for region in regions:
    print(region)

Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79699292, 13.27175306)>
radius: 3.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.80051, 13.2711325)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.80004083, 13.271705)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.80209958, 13.27299472)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79884458, 13.27309694)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79855333, 13.27198778)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equinox=J2000.000): (ra, dec) in deg
    (15.79724583, 13.27124722)>
radius: 5.0 arcsec
Region: CircleSkyRegion
center: <SkyCoord (FK5: equino